In [4]:
import bs4
import pandas as pd
import numpy as np
from src.util.util_dicts import nba_teams_post_2000, home_map
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

## Scraping Player logs

#### Oct 23rd Thinking

Need to start with some base functions that will get called a ton, these will include:

- Open, navigate, pull single year player game log (input player, year, output raw HTML table)
- Take HTML table, parse into rough df that needs formatting
- Take rough df and clean, output single year player stats

Once this is ready, create list of players that played since 2000, could be its own table? (player name, first year, last year)

For now lets use LBJ as our first player to put this in practice on

In [5]:
first = "Lebron"
last = "James"

In [7]:
def player_name_to_bbref_code(first, last):
    if len(last) >= 5:
        return "{}/{}{}01".format(last[:1], last[:5], first[:2]).lower()
    else:
        return "{}/{}{}01".format(last[:1], last, first[:2]).lower()

In [ ]:
def get_player_season_games_log(temp_driver, first, last, season):
    player_code = player_name_to_bbref_code(first, last)
    url = f"https://www.basketball-reference.com/players/{player_code}/gamelog/{season}"
    html = get_webpage_html(temp_driver, url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table = soup.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'games')
    df = pd.read_html(str(table))[0]
    return df



In [1]:
def player_year_to_df(team, year):
    driver = webdriver.Chrome(options=options)
    df_raw = getScheduleResultsDF(driver, team, str(year))
    driver.close()
    df_clean = cleanScheduleResultsDF(df_raw, team)
    return df_clean

In [11]:
driver = webdriver.Chrome()

In [23]:
driver = webdriver.Chrome()
player_code = player_name_to_bbref_code(first, last)
url = f"https://www.basketball-reference.com/players/{player_code}/gamelog/{season}"
driver.get(url)
html = driver.page_source
soup = bs4.BeautifulSoup(html, 'html.parser')
table = soup.find(lambda tag: tag.name == 'table' and tag.has_attr('id') and tag['id'] == 'pgl_basic')
df = pd.read_html(str(table))[0]
driver.close()

In [24]:
df

,Rk,G,Date,Age,Tm,Unnamed: 5,Opp,Unnamed: 7,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
0,1,1,2003-10-29,18-303,CLE,@,SAC,L (-14),1,42:50,...,4,6,9,4,0,2,3,25,24.7,-9
1,2,2,2003-10-30,18-304,CLE,@,PHO,L (-9),1,40:21,...,10,12,8,1,0,7,1,21,14.7,-3
2,3,3,2003-11-01,18-306,CLE,@,POR,L (-19),1,39:10,...,4,4,6,2,0,2,3,8,5.0,-21
3,4,4,2003-11-05,18-310,CLE,NaN,DEN,L (-4),1,41:06,...,9,11,7,2,3,2,1,7,11.2,-3
4,5,5,2003-11-07,18-312,CLE,@,IND,L (-1),1,43:44,...,5,5,3,0,0,7,2,23,9.0,-7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81,79,76,2004-04-09,19-101,CLE,@,MIA,L (-15),1,42:13,...,6,7,8,1,0,8,2,24,15.3,-16
82,80,77,2004-04-10,19-102,CLE,NaN,MIA,W (+11),1,31:38,...,2,3,11,0,0,2,3,10,8.5,+14
83,Rk,G,Date,Age,Tm,NaN,Opp,NaN,GS,MP,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,+/-
84,81,78,2004-04-12,19-104,CLE,NaN,MIL,W (+4),1,45:18,...,6,7,9,2,0,1,1,27,24.8,+11
